## Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [17]:
from mlflow.tracking import MlflowClient

mlflow_tracking_uri = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=mlflow_tracking_uri)

In [20]:
client.tracking_uri

'sqlite:///mlflow.db'

In [28]:
client.search_experiments()

[<Experiment: artifact_location=('/workspaces/Mlops-zoomcamp/2. Experiment tracking and model '
  'management/mlruns/1'), creation_time=1720894759584, experiment_id='1', last_update_time=1720894759584, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1720894457871, experiment_id='0', last_update_time=1720894457871, lifecycle_stage='active', name='Default', tags={}>]

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=1,
    filter_string="",
    run_view_type =ViewType.ACTIVE_ONLY ,
    max_results = 5,
    order_by=["metrics.rmse ASC"]
    
)

In [33]:
for run in runs:
    print(f"run id : {run.info.run_id},rmse :{run.data.metrics['rmse']:.4f} ")

run id : 1e89981de3264168867d9ebe1cef0419,rmse :5.1187 
run id : 0d76a399a6284ef2bd4087425d370544,rmse :5.1187 
run id : 3e316bf936604dbf9ad6f48b0d79b863,rmse :5.1305 
run id : b921b7f28d54411aa56e752c87927ab3,rmse :5.1399 
run id : 17e74ba9e0ac4173b310f32d6511fc37,rmse :5.1429 


## Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the experiment nyc-taxi-regressor
Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 3 was created.
Transition the version 3 to "Staging" and adding annotations to it.

In [34]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/workspaces/Mlops-zoomcamp/2. Experiment tracking and model '
 'management/mlruns/1'), creation_time=1720894759584, experiment_id='1', last_update_time=1720894759584, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [36]:
run_id = "bbb2596bf535457da2c053c82e10632b"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1720942727228, current_stage='None', description=None, last_updated_timestamp=1720942727228, name='nyc-taxi-regressor', run_id='bbb2596bf535457da2c053c82e10632b', run_link=None, source=('/workspaces/Mlops-zoomcamp/2. Experiment tracking and model '
 'management/mlruns/1/bbb2596bf535457da2c053c82e10632b/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [52]:
# Get the list of registered models
models = client.search_registered_models()
for model in models:
    print(f"Model name: {model.name}")
    for version in model.latest_versions:
        print(f" - Version: {version.version}, Status: {version.current_stage}")


Model name: nyc-taxi-regressor
 - Version: 3, Status: Staging


In [45]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1720939547558, description='The NYC Taxi Predictor to Trip Duration', last_updated_timestamp=1720942727228, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1720942727228, current_stage='None', description=None, last_updated_timestamp=1720942727228, name='nyc-taxi-regressor', run_id='bbb2596bf535457da2c053c82e10632b', run_link=None, source=('/workspaces/Mlops-zoomcamp/2. Experiment tracking and model '
  'management/mlruns/1/bbb2596bf535457da2c053c82e10632b/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [51]:
# Parameters for the model version to be transitioned
model_name = "nyc-taxi-regressor"
model_version = "3"
new_stage = "Staging"  # Example stages: "None", "Staging", "Production", "Archived"

# Update the model version stage
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

print(f"Model '{model_name}' version '{model_version}' has been transitioned to stage '{new_stage}'.")

Model 'nyc-taxi-regressor' version '3' has been transitioned to stage 'Staging'.


/tmp/ipykernel_13609/3599651092.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [54]:
# update the Description of the model

import datetime

date = datetime.date.today()

# Parameters for the model version to be transitioned
model_name = "nyc-taxi-regressor"
model_version = "3"
new_stage = "Staging"  # Example stages: "None", "Staging", "Production", "Archived"

# Update the model version stage
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The Model Version {model_version} was transitioned to {new_stage} on {date}"
)


<ModelVersion: aliases=[], creation_timestamp=1720942727228, current_stage='Staging', description='The Model Version 3 was transitioned to Staging on 2024-07-14', last_updated_timestamp=1720944236113, name='nyc-taxi-regressor', run_id='bbb2596bf535457da2c053c82e10632b', run_link=None, source=('/workspaces/Mlops-zoomcamp/2. Experiment tracking and model '
 'management/mlruns/1/bbb2596bf535457da2c053c82e10632b/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

## Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

*Note*: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [56]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

#     df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
#     df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [57]:
df = read_dataframe("data/green_tripdata_2021-01.parquet")

In [62]:
client.download_artifacts(run_id=run_id, path='prerocessor', dst_path='.')

'/workspaces/Mlops-zoomcamp/2. Experiment tracking and model management/prerocessor'

In [64]:
import pickle

with open("prerocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

EOFError: Ran out of input